In [2]:
import glob
import itertools
import json
import os
import pandas
import scipy
from scipy import stats
import matplotlib.pyplot as plt


In [3]:
path = "ExperimentData"
dir_list = os.listdir(path)
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

exp_paths = []

session_map = {}
for filename in dir_list:
    if filename.isdigit():
        exp_paths.append(path + "/" + filename + "/ResponsiveAR/Experiment/")
        #session_map[filename] = user_id
        files = os.listdir(path + "/" + filename + "/ResponsiveAR/Experiment/")
        name = files[0]
        index = name.find(".")
        user_id = name[index - 13: index]
        session_map[user_id] = int(filename)

print(exp_paths)

user_group_mapping = {}

for i in range(30, 87):
    user_group_mapping[i] = "A"
    if(i % 2 != 0):
        user_group_mapping[i] = "B"
        
    #special cases
    if(i == 79):
        user_group_mapping[i] = "A"

print(user_group_mapping)

Files and directories in ' ExperimentData ' :
['61', '59', '68', '57', '32', '35', '69', '56', '51', '58', '67', '60', '34', '33', '.DS_Store', '45', '80', '86', '44', '43', '38', '36', '31', '65', '54', '53', '30', '37', '39', '55', '63', '64', '46', '79', '41', '83', '48', '70', '84', '85', '71', '82', '49', '40', '47', '78']
['ExperimentData/61/ResponsiveAR/Experiment/', 'ExperimentData/59/ResponsiveAR/Experiment/', 'ExperimentData/68/ResponsiveAR/Experiment/', 'ExperimentData/57/ResponsiveAR/Experiment/', 'ExperimentData/32/ResponsiveAR/Experiment/', 'ExperimentData/35/ResponsiveAR/Experiment/', 'ExperimentData/69/ResponsiveAR/Experiment/', 'ExperimentData/56/ResponsiveAR/Experiment/', 'ExperimentData/51/ResponsiveAR/Experiment/', 'ExperimentData/58/ResponsiveAR/Experiment/', 'ExperimentData/67/ResponsiveAR/Experiment/', 'ExperimentData/60/ResponsiveAR/Experiment/', 'ExperimentData/34/ResponsiveAR/Experiment/', 'ExperimentData/33/ResponsiveAR/Experiment/', 'ExperimentData/45/Respon

In [4]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return float(fragments[1])

In [5]:
responsiveData = []
experimentEvents = []

def parseFrame(frames, user_id):
    frames_list = []
    for frame in frames:
        frameData = {
            "userID": float,
            "frameNum": int,
            "unixTime": float,
            "timestamp": float,
            "hPos": {"x":float,"y":float,"z":float},
            "hDir": {"x":float,"y":float,"z":float},
            "hRot": {"i":float,"j":float,"k":float},
            "hAngl": {"x":float,"y":float,"z":float},
            "gazeOrigin": {"x":float,"y":float,"z":float},
            "gazeDirection": {"x":float,"y":float,"z":float},
            "rightHandRay": {"x":float,"y":float,"z":float},
            "leftHandRay": {"x":float,"y":float,"z":float},
            "experimentEvents": [],
            "responsiveData": []
        }
        responsiveEvent = frame["responsiveData"]
        experimentEvent = frame["experimentEvents"]
        frameData["userID"] = user_id
        frameData["frameNum"] = frame["frameNum"]
        frameData["unixTime"] = frame["unixTime"]
        frameData["timestamp"] = frame["timestamp"]
        frameData["hPos"] = frame["hPos"]
        frameData["hDir"] = frame["hDir"]
        frameData["hRot"] = frame["hRot"]
        frameData["hAngl"] = frame["hAngl"]
        frameData["gazeOrigin"] = frame["gazeOrigin"]
        frameData["gazeDirection"] = frame["gazeDirection"]
        frameData["rightHandRay"] = frame["rightHandRay"]
        frameData["leftHandRay"] = frame["leftHandRay"]
        frameData["experimentEvents"] = experimentEvent
        frameData["responsiveData"] = responsiveEvent
        frames_list.append(frameData)
        if responsiveEvent:
            responsiveData.append(responsiveEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)
    return frames_list

def parseSession(session, user_id):
    sessionData = {
    "userID": float,
    "numFrames": int,
    "task": str,
    "start_time": float,
    "end_time": float,
    "total_time": float,
    "sessionNumber": int,
    "isResponsive": int,
    "frames": []
    }
    sessionData["userID"] = user_id
    sessionData["numFrames"] = session["numFrames"]
    sessionData["task"] = session["task"]
    sessionData["start_time"] = session["start_time"]
    sessionData["end_time"] = session["end_time"]
    sessionData["total_time"] = session["total_time"]
    sessionData["sessionNumber"] = session["sessionNumber"]
    sessionData["isResponsive"] = session["isResponsive"]
    return sessionData


In [6]:
data = []
frames = []
allsession = []
user_id = 0

for exp in exp_paths:
    experiment_data = {
    "path" : str,
    "sessions": {}
    }
    sessions = {}
    experiment = os.listdir(exp)
    current_path = exp
    experiment_data["path"] = current_path
    experiment.sort(key=parseFileNumber)
    for filename in experiment:
        if filename.endswith(".json"):
            # Prints only text file present in My Folder
            print(filename)
            firstFileFlag = False
            if("Session_0" in filename):
                firstFileFlag = True
            with open(current_path + filename, 'r') as f:
                json_data = json.load(f)
                if(firstFileFlag):
                    user_id = session_map[json_data["obj"]["userID"]]
                    session = parseSession(json_data["obj"]["sessionRecordings"][0], user_id)
                    frame = parseFrame(json_data["obj"]["sessionRecordings"][0]["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
                else:
                    session = parseSession(json_data, user_id)
                    frame = parseFrame(json_data["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
    experiment_data["sessions"] = sessions
    data.append(experiment_data)

#print(data)


ExperimentSession_0_1653325550021.json
ExperimentSession_1_1653325550021.json
ExperimentSession_2_1653325550021.json
ExperimentSession_3_1653325550021.json
ExperimentSession_4_1653325550021.json
ExperimentSession_5_1653325550021.json
ExperimentSession_6_1653325550021.json
ExperimentSession_7_1653325550021.json
ExperimentSession_8_1653325550021.json
ExperimentSession_9_1653325550021.json
ExperimentSession_0_1652910130460.json
ExperimentSession_1_1652910130460.json
ExperimentSession_2_1652910130460.json
ExperimentSession_3_1652910130460.json
ExperimentSession_4_1652910130460.json
ExperimentSession_5_1652910130460.json
ExperimentSession_6_1652910130460.json
ExperimentSession_7_1652910130460.json
ExperimentSession_8_1652910130460.json
ExperimentSession_9_1652910130460.json
ExperimentSession_0_1653072453927.json
ExperimentSession_1_1653072453927.json
ExperimentSession_2_1653072453927.json
ExperimentSession_3_1653072453927.json
ExperimentSession_4_1653072453927.json
ExperimentSession_5_16530

In [7]:
allsession

sessionsUnpacked = list(itertools.chain(allsession))
sessionsDf = pandas.DataFrame(sessionsUnpacked)
sessionsDf = sessionsDf.sort_values(by='sessionNumber')

#sessionsDf = sessionsDf.loc[sessionsDf["sessionNumber"] < 9 ]
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165..."


USER 56 was unable to finish task 7 (when starting from 0) remove their task 8 from data

In [8]:
session_to_drop = sessionsDf.loc[(sessionsDf["userID"] == 56) & (sessionsDf["sessionNumber"] == 7)]
sessionsDf.drop(session_to_drop.index)

session_to_drop

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
77,56,1434,The task is to find the current location on th...,1652899626599,1652899774967,148368,7,False,"[{'userID': 56, 'frameNum': 1741, 'unixTime': ..."


In [9]:
framesUnpacked = list(itertools.chain(*sessionsDf["frames"]))
framesDf = pandas.DataFrame(framesUnpacked)
print(framesDf.shape)
framesDf.head(1)

(144426, 14)


,userID,frameNum,unixTime,timestamp,hPos,hDir,hRot,hAngl,gazeOrigin,gazeDirection,rightHandRay,leftHandRay,experimentEvents,responsiveData
0,61,1,1653325566491,16470,"{'x': -0.018050827085971832, 'y': -0.004202843...","{'x': -0.0035658455453813076, 'y': -0.24633377...","{'x': 0.12413376569747925, 'y': -0.00124909367...","{'x': 14.260669708251953, 'y': 359.78918457031...","{'x': -0.017374971881508827, 'y': -0.004261824...","{'x': -0.0014919827226549387, 'y': -0.24794106...","{'x': -0.0683116614818573, 'y': 0.569282412528...","{'x': 0.0, 'y': 0.0, 'z': 0.0}","[{'unixTime': 1653325566402, 'systemTime': '10...","{'name': '', 'scale': {'x': 0.0, 'y': 0.0, 'z'..."


In [10]:
users = framesDf.loc[framesDf["experimentEvents"].str.len() != 0]
print(users.shape)

experimentUnpacked = list(itertools.chain(*users["experimentEvents"]))
experimentDf = pandas.DataFrame(experimentUnpacked)
experimentDf.head(1)
print(experimentDf.shape)
experimentDf.shape

experimentDf["userID"] = users["userID"]

for i in users.index:
    eventDF = pandas.DataFrame(users["experimentEvents"][i])

    for j in eventDF.index:
        time = eventDF["unixTime"][j]
        userID = users["userID"][i]
        mask = experimentDf["unixTime"] == time
        experimentDf.loc[mask, "userID"] = userID

experimentDf.head(1)

(2649, 14)
(3560, 10)


,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
0,1653325566402,10-06-06-40,True,instruction,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': -0.01604645512998104, 'y': 0.02574505284...",2nd Title,,61.0


In [11]:
startEvents = experimentDf.loc[experimentDf["eventName"] == "start"]
startEvents.head(3)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
1,1653325572982,10-06-12-98,True,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': -0.01604645512998104, 'y': 0.02574505284...",2nd Title,,61.0
3,1652747526727,17-32-06-72,False,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.06302891671657562, 'y': 0.021540082991...",2nd Title,,34.0
5,1653347136313,16-05-36-31,True,start,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.021247876808047295, 'y': 0.05604220554...",2nd Title,,53.0


In [12]:
sessionsDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165..."
120,34,210,The task is to click on the 2nd article title....,1652747526667,1652747540607,13940,0,False,"[{'userID': 34, 'frameNum': 1, 'unixTime': 165..."
240,53,153,The task is to click on the 2nd article title....,1653347136263,1653347142364,6101,0,True,"[{'userID': 53, 'frameNum': 1, 'unixTime': 165..."
330,41,80,The task is to click on the 2nd article title....,1652904601496,1652904604747,3251,0,True,"[{'userID': 41, 'frameNum': 1, 'unixTime': 165..."
388,71,110,The task is to click on the 2nd article title....,1653342438648,1653342442018,3370,0,True,"[{'userID': 71, 'frameNum': 1, 'unixTime': 165..."
...,...,...,...,...,...,...,...,...,...
369,70,1,,0,0,0,9,False,"[{'userID': 70, 'frameNum': 2210, 'unixTime': ..."
119,60,1,,0,0,0,9,False,"[{'userID': 60, 'frameNum': 3456, 'unixTime': ..."
239,54,1,,0,0,0,9,False,"[{'userID': 54, 'frameNum': 2705, 'unixTime': ..."
39,57,1,,0,0,0,9,False,"[{'userID': 57, 'frameNum': 3809, 'unixTime': ..."


In [13]:
sessionsDf["Actual_Start_Time"] = sessionsDf["start_time"]

for i in startEvents.index:
    userID = startEvents["userID"][i]
    task_num = startEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_Start_Time"] = startEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982


In [14]:
endEvents = experimentDf.loc[experimentDf["eventName"] == "complete"]
endEvents.head(1)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
94,1652899453671,11-44-13-67,False,complete,0,Article Display,"{'x': 0.25, 'y': 0.25, 'z': 0.25}","{'x': 0.21994057297706604, 'y': -0.01243835873...",2nd Title,2nd Title,56.0


In [15]:
sessionsDf["Actual_End_Time"] = sessionsDf["end_time"]

for i in endEvents.index:
    userID = endEvents["userID"][i]
    task_num = endEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_End_Time"] = endEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982,1653325575801


In [16]:
end_time_diff = sessionsDf["end_time"].to_numpy() - sessionsDf["Actual_End_Time"].to_numpy()
end_time_diff

array([  0, -10,   0,   0,   0, -10,   0, -11,   0,   0,   0,   0,  -3,
       -11,   0,   0,   0, -10,   0,   0,  -4, -11,   0, -10,  -7, -10,
       -10, -10,   0,   0, -10, -11,   0, -10,   0, -10, -10, -10,  -5,
         0,   0,   0,   0,   0, -10,   0,   0,   0,   0, -10, -10,   0,
       -12, -10,   0,   0,  -9,   0,   0,   0,   0, -10, -11,   0, -10,
         0,   0, -10,   0,  -6,   0, -10,   0, -10, -10,   0, -11, -10,
        -4,   0,   0,   0,   0, -10,   0,  -2,   0,   0,   0,   0, -10,
        -9,   0,   0, -10,   0,   0,  -9,   0,   0,   0,  -6,   0,   0,
         0, -10,   0,   0,   0,   0,   0,  -8, -10,   0,   0,   0, -10,
        -5, -10,   0,   0,   0,   0, -11, -10,   0,  -7,  -6,   0,  -4,
         0,   0,   0,   0, -10,   0,   0, -10,   0,   0, -10,   0,   0,
        -4,   0,   0,   0, -10,   0,   0,   0,   0,   0,   0, -10,   0,
       -10, -10, -11,  -7,  -3,   0, -10, -10,  -9,  -6, -10,  -9,  -9,
         0, -10, -10, -10,   0,   0,   0,  -9, -10,   0, -10,   

In [17]:
guessEvents = experimentDf.loc[experimentDf["eventName"] == "answer"]
guessEvents.head(3)

sessionsDf["Guesses"] = 0

In [18]:
for i in guessEvents.index:
    userID = guessEvents["userID"][i]
    task_num = guessEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Guesses"] += 1

In [19]:
start_time_diff = sessionsDf["start_time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
start_time_diff

array([           -50,            -60,            -50,            -50,
                  -50,            -60,            -50,            -56,
                  -50,            -50,            -50,            -61,
                  -50,            -50,            -60,            -56,
                  -50,            -61,            -60,            -51,
                  -60,            -50,            -49,            -52,
                  -62,            -60,            -59,            -50,
                  -60,            -49,            -51,            -61,
                  -50,            -50,            -50,            -50,
                  -43,            -60,            -60,            -45,
                  -60,            -50,            -60,            -51,
                  -60,            -45,            -40,            -30,
                  -40,            -40,            -43,            -40,
                  -50,            -40,            -38,            -39,
      

In [20]:
sessionsDf["Actual_Total_Time"] = sessionsDf["Actual_End_Time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Actual_Total_Time
0,61,92,The task is to click on the 2nd article title....,1653325572932,1653325575801,2869,0,True,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",1653325572982,1653325575801,1,2819


In [21]:
start_time_diff = sessionsDf["total_time"].to_numpy() - sessionsDf["Actual_Total_Time"].to_numpy()
start_time_diff

array([           50,            50,            50,            50,
                  50,            50,            50,            45,
                  50,            50,            50,            61,
                  47,            39,            60,            56,
                  50,            51,            60,            51,
                  56,            39,            49,            42,
                  55,            50,            49,            40,
                  60,            49,            41,            50,
                  50,            40,            50,            40,
                  33,            50,            55,            45,
                  60,            50,            60,            51,
                  50,            45,            40,            30,
                  40,            30,            33,            40,
                  38,            30,            38,            39,
                  31,            40,            40,           

In [22]:
responsiveDf = sessionsDf.loc[sessionsDf["isResponsive"] == True]
responsiveDf = responsiveDf.sort_values("Actual_Total_Time")
responsiveDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Actual_Total_Time
131,33,53,The task is to click the 5th article title.\n\...,1652743242306,1652743244466,2160,1,True,"[{'userID': 33, 'frameNum': 95, 'unixTime': 16...",1652743242346,1652743244466,1,2120
261,37,44,The task is to click the 5th article title.\n\...,1652818011143,1652818013353,2210,1,True,"[{'userID': 37, 'frameNum': 53, 'unixTime': 16...",1652818011183,1652818013353,1,2170
211,31,67,The task is to click the 5th article title.\n\...,1652726021550,1652726023839,2289,1,True,"[{'userID': 31, 'frameNum': 189, 'unixTime': 1...",1652726021589,1652726023839,1,2250
1,61,48,The task is to click the 5th article title.\n\...,1653325578382,1653325580702,2320,1,True,"[{'userID': 61, 'frameNum': 93, 'unixTime': 16...",1653325578422,1653325580702,1,2280
63,69,118,The task is to click the Date of the 1st artic...,1653438698298,1653438700788,2490,3,True,"[{'userID': 69, 'frameNum': 362, 'unixTime': 1...",1653438698338,1653438700798,1,2460
...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,31,1719,The task is to type phone number 805-UCSB-EDU ...,1652726084621,1652726272953,188332,5,True,"[{'userID': 31, 'frameNum': 806, 'unixTime': 1...",1652726084621,1652726272953,4,188332
246,53,1921,The task is to find the current temperature on...,1653347322034,1653347533250,211216,6,True,"[{'userID': 53, 'frameNum': 1803, 'unixTime': ...",1653347322037,1653347533250,23,211213
185,43,2221,The task is to type phone number 805-UCSB-EDU ...,1652990824367,1652991077151,252784,5,True,"[{'userID': 43, 'frameNum': 1430, 'unixTime': ...",1652990824377,1652991077162,3,252785
288,55,2324,The task is to find the low temperature on th...,1653526723809,1653526987514,263705,8,True,"[{'userID': 55, 'frameNum': 2672, 'unixTime': ...",1653526723819,1653526987514,68,263695


In [23]:
nonresponsiveDf = sessionsDf.loc[sessionsDf["isResponsive"] == False]
nonresponsiveDf = nonresponsiveDf.sort_values("Actual_Total_Time")
nonresponsiveDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Actual_Total_Time
387,85,3928,,0,0,0,7,False,"[{'userID': 85, 'frameNum': 1877, 'unixTime': ...",1653699433552,0,101,-1653699433552
447,78,1,,0,0,0,9,False,"[{'userID': 78, 'frameNum': 2803, 'unixTime': ...",0,0,0,0
129,34,1,,0,0,0,9,False,"[{'userID': 34, 'frameNum': 4089, 'unixTime': ...",0,0,0,0
69,69,1,,0,0,0,9,False,"[{'userID': 69, 'frameNum': 2353, 'unixTime': ...",0,0,0,0
319,46,1,,0,0,0,9,False,"[{'userID': 46, 'frameNum': 2587, 'unixTime': ...",0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,46,722,The task is to find the current location on th...,1653088505254,1653088645361,140107,7,False,"[{'userID': 46, 'frameNum': 1682, 'unixTime': ...",1653088505259,1653088645367,37,140108
77,56,1434,The task is to find the current location on th...,1652899626599,1652899774967,148368,7,False,"[{'userID': 56, 'frameNum': 1741, 'unixTime': ...",1652899626604,1652899774967,45,148363
128,34,1464,The task is to find the low temperature on th...,1652747802011,1652747952843,150832,8,False,"[{'userID': 34, 'frameNum': 2625, 'unixTime': ...",1652747802021,1652747952843,46,150822
118,60,2011,The task is to find the low temperature on th...,1653093267183,1653093491077,223894,8,False,"[{'userID': 60, 'frameNum': 1445, 'unixTime': ...",1653093267194,1653093491077,56,223883


In [24]:
#task 0
i = 0
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))



TASK 0: The task is to click on the 2nd article title.

When you are ready, click the 'Start Task' button.
responsive task 0 Time: 5877.391 ± 2746.772
normal task 0 Time: 8732.864 ± 9499.073
Ttest_indResult(statistic=-1.3831090909158037, pvalue=0.1737714324941954)


In [25]:
#Task 1
i = 1
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 1: The task is to click the 5th article title.

When you are ready, click the 'Start Task' button.
responsive task 1 Time: 5203.478 ± 3872.201
normal task 1 Time: 13870.909 ± 12837.267
Ttest_indResult(statistic=-3.0955747890442433, pvalue=0.003450230223960752)


In [26]:
#Task 2
i = 2
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))


TASK 2: The task is to click the Author of the 4th article.

When you are ready, click the 'Start Task' button.
responsive task 2 Time: 15490.261 ± 17195.831
normal task 2 Time: 10940.682 ± 8141.135
Ttest_indResult(statistic=1.1257419043396346, pvalue=0.2665188548589997)


In [27]:
#task 3
i = 3
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))


TASK 3: The task is to click the Date of the 1st article.

When you are ready, click the 'Start Task' button.
responsive task 3 Time: 8959.870 ± 6267.435
normal task 3 Time: 11594.545 ± 7113.611
Ttest_indResult(statistic=-1.319795354487707, pvalue=0.19388696897042312)


In [28]:
#task 4
i = 4
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))


TASK 4: The task is to type the phone number 949-927-4580 on the touchpad then click call. 

When you are ready, click the 'Start Task' button.
responsive task 4 Time: 28854.957 ± 13266.773
normal task 4 Time: 24918.045 ± 9271.555
Ttest_indResult(statistic=1.1489093643638997, pvalue=0.2569442461906608)


In [29]:
#task 5
i = 5
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 5: The task is to type phone number 805-UCSB-EDU on the touchpad then click Call. 

When you are ready, click the 'Start Task' button.
responsive task 5 Time: 53541.957 ± 59750.487
normal task 5 Time: 31121.727 ± 22549.226
Ttest_indResult(statistic=1.6504828442201085, pvalue=0.10613076892992643)


In [30]:
#task 6
i = 6
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))



TASK 6: The task is to find the current temperature on the weather application and type it into the keyboard. 

When you are ready, click the 'Start Task' button.
responsive task 6 Time: 56729.478 ± 89441.012
normal task 6 Time: 26114.273 ± 13415.439
Ttest_indResult(statistic=1.5877367377732825, pvalue=0.11967290628707869)


In [31]:
#task 7
i = 7
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 7: The task is to find the current location on the weather application and type it into the keyboard. The answer is not case sensitive.

When you are ready, click the 'Start Task' button.
responsive task 7 Time: 59959.000 ± 30794.537
normal task 7 Time: -71899916539.174 ± 344820181238.686
Ttest_indResult(statistic=0.9775251988634808, pvalue=0.3337769726486889)


In [32]:
#task 8
i = 8
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["isResponsive"] == True]
task_n = taskDf.loc[taskDf["isResponsive"] == False]

print(f"TASK {i}: " + task_r["task"][i])

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 8: The task is to find the low temperature  on the weather application and type it into the keyboard. 

When you are ready, click the 'Start Task' button.
responsive task 8 Time: 49416.727 ± 65716.578
normal task 8 Time: 43658.273 ± 54653.853
Ttest_indResult(statistic=0.31599921130774755, pvalue=0.7535681794417899)


In [33]:
#each task in a loop
for i in range(0,9):
    taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
    taskDf
    task_r = taskDf.loc[taskDf["isResponsive"] == True]
    task_n = taskDf.loc[taskDf["isResponsive"] == False]

    print(f"TASK {i}: " + task_r["task"][i])

    taskRAvgTime = task_r['Actual_Total_Time'].mean()
    taskRStd = task_r['Actual_Total_Time'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Actual_Total_Time'].mean()
    taskNStd = task_n['Actual_Total_Time'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

TASK 0: The task is to click on the 2nd article title.

When you are ready, click the 'Start Task' button.
responsive task 0 Time: 5877.391 ± 2746.772
normal task 0 Time: 8732.864 ± 9499.073
Ttest_indResult(statistic=-1.3831090909158037, pvalue=0.1737714324941954)
TASK 1: The task is to click the 5th article title.

When you are ready, click the 'Start Task' button.
responsive task 1 Time: 5203.478 ± 3872.201
normal task 1 Time: 13870.909 ± 12837.267
Ttest_indResult(statistic=-3.0955747890442433, pvalue=0.003450230223960752)
TASK 2: The task is to click the Author of the 4th article.

When you are ready, click the 'Start Task' button.
responsive task 2 Time: 15490.261 ± 17195.831
normal task 2 Time: 10940.682 ± 8141.135
Ttest_indResult(statistic=1.1257419043396346, pvalue=0.2665188548589997)
TASK 3: The task is to click the Date of the 1st article.

When you are ready, click the 'Start Task' button.
responsive task 3 Time: 8959.870 ± 6267.435
normal task 3 Time: 11594.545 ± 7113.611
Tt

In [34]:
for i in range(0,9):
    print(f"TASK {str(i)}:")
    taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
    taskDf
    task_r = taskDf.loc[taskDf["isResponsive"] == True]
    task_n = taskDf.loc[taskDf["isResponsive"] == False]

    taskRAvgTime = task_r['Guesses'].mean()
    taskRStd = task_r['Guesses'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['Guesses'].mean()
    taskNStd = task_n['Guesses'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['Guesses'], task_n['Guesses']))

TASK 0:
responsive task 0 Time: 1.043 ± 0.209
normal task 0 Time: 1.091 ± 0.294
Ttest_indResult(statistic=-0.626117235046913, pvalue=0.5345477403968492)
TASK 1:
responsive task 1 Time: 1.130 ± 0.458
normal task 1 Time: 1.500 ± 0.859
Ttest_indResult(statistic=-1.8121843206506378, pvalue=0.0769443111920115)
TASK 2:
responsive task 2 Time: 1.739 ± 1.738
normal task 2 Time: 2.000 ± 2.160
Ttest_indResult(statistic=-0.4473318277810719, pvalue=0.6568808306601277)
TASK 3:
responsive task 3 Time: 1.217 ± 0.422
normal task 3 Time: 1.727 ± 1.120
Ttest_indResult(statistic=-2.0375970078942585, pvalue=0.04777340937857098)
TASK 4:
responsive task 4 Time: 1.043 ± 0.209
normal task 4 Time: 1.091 ± 0.294
Ttest_indResult(statistic=-0.626117235046913, pvalue=0.5345477403968492)
TASK 5:
responsive task 5 Time: 1.261 ± 0.752
normal task 5 Time: 1.045 ± 0.213
Ttest_indResult(statistic=1.2944933360701294, pvalue=0.2024037595750517)
TASK 6:
responsive task 6 Time: 8.957 ± 12.719
normal task 6 Time: 4.909 ± 5.3

In [35]:
usersDf = sessionsDf[["userID"]]
usersDf = usersDf.drop_duplicates()
usersDf = usersDf.sort_values("userID")
usersDf

,userID
250,30
210,31
40,32
130,33
120,34
50,35
200,36
260,37
190,38
270,39


In [36]:
allTasksDf = usersDf

In [37]:
for i in range(0,9):
    allTasksDf[f"task{i}"] = i

allTasksDf["all_tasks"] = 0
allTasksDf["group"] = "A"
allTasksDf

,userID,task0,task1,task2,task3,task4,task5,task6,task7,task8,all_tasks,group
250,30,0,1,2,3,4,5,6,7,8,0,A
210,31,0,1,2,3,4,5,6,7,8,0,A
40,32,0,1,2,3,4,5,6,7,8,0,A
130,33,0,1,2,3,4,5,6,7,8,0,A
120,34,0,1,2,3,4,5,6,7,8,0,A
50,35,0,1,2,3,4,5,6,7,8,0,A
200,36,0,1,2,3,4,5,6,7,8,0,A
260,37,0,1,2,3,4,5,6,7,8,0,A
190,38,0,1,2,3,4,5,6,7,8,0,A
270,39,0,1,2,3,4,5,6,7,8,0,A


In [38]:
for i in allTasksDf.index:
    user = allTasksDf["userID"][i]
    userSessions = sessionsDf.loc[sessionsDf["userID"] == user]
    total = 0
    task = userSessions.loc[userSessions["sessionNumber"] == 0]
    allTasksDf.loc[i, "task0"] = task["Actual_Total_Time"].to_numpy()[0]
    total = int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 1]
    allTasksDf.loc[i, "task1"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 2]
    allTasksDf.loc[i, "task2"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 3]
    allTasksDf.loc[i, "task3"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 4]
    allTasksDf.loc[i, "task4"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 5]
    allTasksDf.loc[i, "task5"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 6]
    allTasksDf.loc[i, "task6"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 7]
    allTasksDf.loc[i, "task7"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    task = userSessions.loc[userSessions["sessionNumber"] == 8]
    allTasksDf.loc[i, "task8"] = task["Actual_Total_Time"].to_numpy()[0]
    total += int(task["Actual_Total_Time"])
    allTasksDf.loc[i, "all_tasks"] = total
    allTasksDf.loc[i, "group"] = user_group_mapping[user]

#allTasksDf = allTasksDf.drop(columns=["task{i}"])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [39]:
allTasksDf = allTasksDf.sort_values("all_tasks")
allTasksDf

,userID,task0,task1,task2,task3,task4,task5,task6,task7,task8,all_tasks,group
160,86,0,1,2,3,4,5,6,7,8,0,A
380,85,3510,2700,19690,10280,26301,13149,71291,-1653699433552,8,0,A
0,61,2819,2280,2750,2983,18900,13570,10451,32670,10160,96583,B
200,36,3250,7150,9881,5910,13390,12400,17100,17020,10820,96921,A
40,32,3994,4471,2915,7051,16741,28440,10321,39910,7954,121797,A
360,70,4574,10060,12190,6031,25960,15810,18989,27610,10470,131694,A
370,84,6516,7681,2728,6650,15920,19761,24130,41441,12950,137777,A
428,47,4050,3460,14081,13530,26840,21502,12014,31820,16470,143767,B
190,38,4560,8668,15541,6050,35881,18399,20451,27610,13329,150489,A
90,58,4340,11560,11071,16220,22540,38590,20421,20420,10938,156100,A


In [40]:
groupA = allTasksDf.loc[allTasksDf["group"] == "A"]
groupB = allTasksDf.loc[allTasksDf["group"] == "B"]

groupATime = groupA['all_tasks'].mean()
groupAStd = groupA['all_tasks'].std()
print(f"group A Time: {groupATime:.3f} ± {groupAStd:.3f}")

groupBTime = groupB['all_tasks'].mean()
groupBStd = groupB['all_tasks'].std()
print(f"group B Time: {groupBTime:.3f} ± {groupBStd:.3f}")

print(scipy.stats.ttest_ind(groupB['all_tasks'], groupA['all_tasks']))

group A Time: 212920.652 ± 123509.639
group B Time: 285293.864 ± 135791.513
Ttest_indResult(statistic=1.8718179835834678, pvalue=0.06804391026859603)


In [41]:
sessionsDf.loc[(sessionsDf["userID"] == 30) & (sessionsDf["sessionNumber"] == 0)]

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,Actual_Start_Time,Actual_End_Time,Guesses,Actual_Total_Time
250,30,240,The task is to click on the 2nd article title....,1652721816735,1652721833975,17240,0,False,"[{'userID': 30, 'frameNum': 1, 'unixTime': 165...",1652721816794,1652721833985,1,17191


In [ ]:
groupA


,userID,task0,task1,task2,task3,task4,task5,task6,task7,task8,all_tasks,group
160,86,0,1,2,3,4,5,6,7,8,0,A
380,85,3510,2700,19690,10280,26301,13149,71291,-1653699433552,8,0,A
200,36,3250,7150,9881,5910,13390,12400,17100,17020,10820,96921,A
40,32,3994,4471,2915,7051,16741,28440,10321,39910,7954,121797,A
360,70,4574,10060,12190,6031,25960,15810,18989,27610,10470,131694,A
370,84,6516,7681,2728,6650,15920,19761,24130,41441,12950,137777,A
190,38,4560,8668,15541,6050,35881,18399,20451,27610,13329,150489,A
90,58,4340,11560,11071,16220,22540,38590,20421,20420,10938,156100,A
250,30,17191,12730,17180,9970,21701,16280,16921,41531,9373,162877,A
320,79,6347,15310,17220,11180,18721,25611,12180,42901,29821,179291,A


In [42]:
surveyDf = pandas.read_csv("PostStudy.csv")
surveyDf["group"] = 0

for i in surveyDf.index:
    mask = surveyDf.index == i
    surveyDf["group"][i] = user_group_mapping[surveyDf["What is your Participant Id?"][i]]


<ipython-input-42-bd0d7fad6bc9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surveyDf["group"][i] = user_group_mapping[surveyDf["What is your Participant Id?"][i]]
/Users/abbywysopal/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-42-bd0d7fad6bc9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surveyD

In [ ]:
groupA = surveyDf.loc[surveyDf["group"] == "A"]
groupA = groupA[groupA.columns[9]]

groupA


0     1
2     5
4     3
5     3
7     2
10    6
11    6
12    3
16    6
18    1
21    1
22    7
23    6
24    1
29    2
31    4
32    1
35    3
38    6
39    2
40    2
41    5
Name: It was easy to read with the applications., dtype: int64

In [43]:
for i in range(9,19):
    groupA = surveyDf.loc[surveyDf["group"] == "A"]
    groupB = surveyDf.loc[surveyDf["group"] == "B"]

    groupAresponseDf = groupA[groupA.columns[i]]
    groupBresponseDf = groupB[groupB.columns[i]]

    groupBResponse = groupBresponseDf.mean()
    groupBStd = groupBresponseDf.std()
    print(groupA.columns[i])
    print(f"group B {str(i)}: {groupBResponse:.3f} ± {groupBStd:.3f}")

    groupAResponse = groupAresponseDf.mean()
    groupAStd = groupAresponseDf.std()
    print(f"group A {str(i)}: {groupAResponse:.3f} ± {groupAStd:.3f}")

    print(scipy.stats.ttest_ind(groupBresponseDf, groupAresponseDf))
    print(scipy.stats.wilcoxon(groupBresponseDf, groupAresponseDf))

It was easy to read with the applications.
group B 9: 2.864 ± 1.356
group A 9: 3.455 ± 2.064
Ttest_indResult(statistic=-1.1224308019732516, pvalue=0.2680551070997155)
WilcoxonResult(statistic=82.0, pvalue=0.3861533557024709)
It was easy to interact with the applications.
group B 10: 2.955 ± 1.558
group A 10: 3.864 ± 1.885
Ttest_indResult(statistic=-1.743925009515161, pvalue=0.08848810921794373)
WilcoxonResult(statistic=51.0, pvalue=0.12955906264524883)
It was enjoyable to interact with the applications.
group B 11: 2.909 ± 1.342
group A 11: 3.455 ± 1.920
Ttest_indResult(statistic=-1.0919839843523433, pvalue=0.28106467129799034)
WilcoxonResult(statistic=52.5, pvalue=0.25195216311281854)
The tasks were easy to understand
group B 12: 1.818 ± 1.296
group A 12: 2.273 ± 2.208
Ttest_indResult(statistic=-0.8327827331190546, pvalue=0.40967798021769397)
WilcoxonResult(statistic=46.0, pvalue=0.6766180774863007)
It was easy to complete the tasks.
group B 13: 2.273 ± 1.453
group A 13: 2.955 ± 1.889

/Users/abbywysopal/miniconda3/lib/python3.8/site-packages/scipy/stats/morestats.py:2967: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
